In [ ]:
'''
Deep Learning Programming Assignment 2
--------------------------------------
Name: Vishnu Dutt Sharma
Roll No.: 12EC35018


======================================

Problem Statement:
This assignment has two problems.

P1. Implement a simple 1 or 2 hidden layer MLP USING any deep learning library
for predicting MNIST images.
P2. Implement a simple CNN USING any deep learning library
for predicting MNIST images.

Resources:
1. https://www.tensorflow.org/get_started/mnist/beginners
2. https://www.tensorflow.org/get_started/mnist/pros

======================================

Instructions:
1. Download the MNIST dataset from http://yann.lecun.com/exdb/mnist/
    (four files).
2. Extract all the files into a folder named `data' just outside
    the folder containing the main.py file. This code reads the
    data files from the folder '../data'.
3. Complete the functions in the train_dense.py and train_cnn.py files. You might also
    create other functions for your convenience, but do not change anything
    in the main.py file or the function signatures of the train and test
    functions in the train files.
4. The train function must train the neural network given the training
    examples and save the in a folder named `weights' in the same
    folder as main.py
5. The test function must read the saved weights and given the test
    examples it must return the predicted labels.
6. Submit your project folder with the weights. Note: Don't include the
    data folder, which is anyway outside your project folder.

Submission Instructions:
1. Fill your name and roll no in the space provided above.
2. Name your folder in format <Roll No>_<First Name>.
    For example 12CS10001_Rohan
3. Submit a zipped format of the file (.zip only).
'''

In [1]:
import numpy as np
import os
import train_dense
import train_cnn
import tensorflow as tf
from sklearn.model_selection import train_test_split
import random
import pickle

In [2]:
def multilayer_perceptron(x, weights, biases):
    # Hidden layer with RELU activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    # Hidden layer with RELU activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    # Output layer with linear activation
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer


def train_nn(trainX, trainY):
    
    trainX_mod = trainX.reshape((trainX.shape[0], trainX.shape[1]*trainX.shape[2]))
    y_mod = []
    for i in trainY:
        zer = [0]*10
        zer[i] = 1
        y_mod.append(zer)

    trainY_mod = np.asarray(y_mod)
    
    n_hidden = 400
    n_input = 784
    n_classes = 10

    n_hidden_1 = 400  # 1st layer number of features
    n_hidden_2 = 100# 2nd layer number of features
    n_input = trainX_mod.shape[1] # MNIST data input (img shape: 28*28)
    n_classes = trainY_mod.shape[1] # MNIST total classes (0-9 digits)

    
    
    
    X = tf.placeholder("float", shape=[None, n_input])
    y = tf.placeholder("float", shape=[None, n_classes])

    # Store layers weight & bias
    weights = {
        'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
        'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
        'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden_1])),
        'b2': tf.Variable(tf.random_normal([n_hidden_2])),
        'out': tf.Variable(tf.random_normal([n_classes]))
    }
    
    x = tf.placeholder(tf.float32, shape=[None, n_input])
    y_ = tf.placeholder(tf.float32, [None, 10])
    
    predict = multilayer_perceptron(X, weights, biases)

    # Define loss and optimizer
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=predict, labels=y))
    updates = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)
    
    
    

    sess = tf.Session()
    init = tf.global_variables_initializer()
    sess.run(init)

    
    accuracy = 0.0
    batch_size = 1000
    iterat = 50
    
    for epoch in range(50):
        train_X, test_X, train_y, test_y = train_test_split(trainX_mod, trainY_mod, test_size=0.20, random_state=random.randint(1,99))
        # Train with each example
        for i in range(len(train_X)/batch_size):
            sess.run(updates, feed_dict={X: train_X[i*batch_size: (i + 1)*batch_size], y: train_y[i*batch_size: (i + 1)*batch_size]})

        train_accuracy = np.mean(np.argmax(train_y, axis=1) ==
                                 np.argmax(sess.run(predict, feed_dict={X: train_X, y: train_y}), axis=1))
        test_accuracy  = np.mean(np.argmax(test_y, axis=1) ==
                                 np.argmax(sess.run(predict, feed_dict={X: test_X, y: test_y}), axis=1))

        accuracy += test_accuracy
        print("Epoch = %d, train accuracy = %.2f%%, test accuracy = %.2f%%"
              % (epoch + 1, 100. * train_accuracy, 100. * test_accuracy))
    
    
    
    
    w1= sess.run(weights['h1'])
    np.save('./output/weight_nn_1', w1)

    w2= sess.run(weights['h2'])
    np.save('./output/weight_nn_2', w2)
    
    wout= sess.run(weights['out'])
    np.save('./output/weight_nn_out', wout)

    b1= sess.run(biases['b1'])
    np.save('./output/bias_nn_1', b1)

    b2= sess.run(biases['b2'])
    np.save('./output/bias_nn_2', b2)
    
    bout= sess.run(biases['out'])
    np.save('./output/bias_nn_out', bout)

    
#     w1= sess.run(weights['h1'])
#     np.save('weight1', w1)
    
#     w2= sess.run(weights['out'])
#     np.save('weight2', w2)
    
#     b1= sess.run(biases['b1'])
#     np.save('bias1', b1)
    
#     b2= sess.run(biases['out'])
#     np.save('bias2', b2)
    
    
    sess.close()
    '''
    Complete this function.
    '''

def test_nn(testX):
    testX_mod = testX.reshape((testX.shape[0], testX.shape[1]*testX.shape[2]))
    
    n_hidden = 400
    n_input = 784
    n_classes = 10

    n_hidden_1 = 500  # 1st layer number of features
    n_hidden_2 = 150# 2nd layer number of features
    n_input = testX_mod.shape[1] # MNIST data input (img shape: 28*28)
    
    
    
    
    X = tf.placeholder("float", shape=[None, n_input])
    
    weights = dict()
    biases = dict()
    
    weights['h1'] = np.load('./output/weight_nn_1.npy')
    weights['h2'] = np.load('./output/weight_nn_2.npy')
    weights['out'] = np.load('./output/weight_nn_out.npy')

    biases['b1'] = np.load('./output/bias_nn_1.npy')
    biases['b2'] = np.load('./output/bias_nn_2.npy')
    biases['out'] = np.load('./output/bias_nn_out.npy')
    
  
    x = tf.placeholder(tf.float32, shape=[None, n_input])
    
    predict = multilayer_perceptron(X, weights, biases)
    
    sess = tf.Session()
    init = tf.global_variables_initializer()
    sess.run(init)
    
    output = np.argmax(sess.run(predict, feed_dict={X: testX_mod}), axis=1)
    
    return output
    

In [3]:
## Convolutional Neural Network

RANDOM_SEED = 42
tf.set_random_seed(RANDOM_SEED)

#function to convert matrix to vector
def cnn_convertToVector(matrix):   
    vector = matrix.reshape((matrix.shape[0], matrix.shape[1]*matrix.shape[2])).astype(np.float)
    vector = vector - vector.mean(axis=0)
    return vector

def cnn_InitWeights(shape):
    weight = tf.random_normal(shape, stddev = 0.1)
    weight = tf.cast(weight, tf.float32) 
    return tf.Variable(weight)

def cnn_InitBias(shape):
    bias = tf.constant(1, shape = shape)
    bias = tf.cast(bias, tf.float32) 
    return tf.Variable(bias)

def cnn_ForwardPropogation(X, w_1, b_1, w_2, b_2, w_fc1, b_fc1, w_fc2, b_fc2):
    X = tf.reshape(X, [-1,28,28,1])
    h_conv1 = tf.nn.sigmoid(tf.nn.conv2d(X, w_1, strides = [1,1,1,1], padding = 'SAME') + b_1)
    h_pool1 = tf.nn.max_pool(h_conv1, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'SAME')
    
    h_conv2 = tf.nn.sigmoid(tf.nn.conv2d(h_pool1, w_2, strides = [1,1,1,1], padding = 'SAME') + b_2)
    h_pool2 = tf.nn.max_pool(h_conv2, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'SAME', )
    
    h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])

    h_fc1 = tf.nn.sigmoid(tf.matmul(h_pool2_flat, w_fc1) + b_fc1)
    y_conv = tf.matmul(h_fc1, w_fc2) + b_fc2
    return y_conv

def cnn_createData(X, Y):
    X = cnn_convertToVector(X)
    num_labels = len(np.unique(Y))
    Y_temp = np.eye(num_labels)[Y] 
    return X, Y_temp
    
def train_cnn_train(trainX, trainY):
    
    # Symbols
    X_all,Y_all  = cnn_createData(trainX, trainY)
    train_X, test_X, train_y, test_y = train_test_split(X_all, Y_all, test_size=0.10, random_state=RANDOM_SEED)    
    
    print("Train Size: ")
    print(train_X.shape)
    print("Test Size: ")
    print(test_X.shape)
    
    x_size = train_X.shape[1] 
    y_size = train_y.shape[1]
    X = tf.placeholder("float32", shape=[None, x_size])
    y = tf.placeholder("float32", shape=[None, y_size])
    
    # Weight & bias initializations
    w_1 = cnn_InitWeights([4,4,1,32])
    b_1 = cnn_InitBias([32])
    w_2 = cnn_InitWeights([4,4,32,64])
    b_2 = cnn_InitBias([64])
    w_fc1 = cnn_InitWeights([64*7*7, 1024])
    b_fc1 = cnn_InitBias([1024])
    w_fc2 = cnn_InitWeights([1024, 10])
    b_fc2 = cnn_InitBias([10])
     
    # Forward propagation
    yhat = cnn_ForwardPropogation(X, w_1, b_1, w_2, b_2, w_fc1, b_fc1, w_fc2, b_fc2)
    predict = tf.argmax(yhat, axis=1)
    
    # Back propogation
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = y, logits = yhat))
    updates = tf.train.AdamOptimizer(1e-3).minimize(cost)
    
     # Run SGD    
    sess = tf.Session()
    init = tf.global_variables_initializer()
    sess.run(init)
    
    num_epoch = 1
    data_length = train_X.shape[0] 
    batch_size = 5000
    number_batches = int(np.ceil(data_length/batch_size))    
        
    for epoch in range(num_epoch):
        batches = list(range(number_batches))
        random.shuffle(batches)
        for batch_index in batches:
            if((batch_index+1)*batch_size < data_length):
                batch_X = train_X[batch_index*batch_size:(batch_index+1)*batch_size]
                batch_Y = train_y[batch_index*batch_size:(batch_index+1)*batch_size]
            else:
                batch_X = train_X[batch_index*batch_size:data_length]
                batch_Y = train_y[batch_index*batch_size:data_length]

            # Train with each example
            train_accuracy = 0
            for i in range(len(batch_X)):
                sess.run(updates, feed_dict={X: batch_X[i: i + 1], y: batch_Y[i: i + 1]})
                if(np.argmax(batch_Y[i: i + 1], axis=1) == sess.run(predict, feed_dict={X: batch_X[i: i + 1], y: batch_Y[i: i + 1]})):
                    train_accuracy += 1       
            train_accuracy /= len(batch_X)
            
            test_accuracy = 0
            for i in range(len(test_y)):
                if(np.argmax(test_y[i: i + 1], axis=1) == sess.run(predict, feed_dict={X: test_X[i: i + 1], y: test_y[i: i + 1]})):
                    test_accuracy += 1
            test_accuracy /= len(test_y)
            
#             train_accuracy = np.mean(np.argmax(batch_Y, axis=1) == sess.run(predict, feed_dict={X: batch_X[i: i + 1], y: batch_Y[i: i + 1]}))
#             test_accuracy = 0
#             test_accuracy  = np.mean(np.argmax(test_y, axis=1) == sess.run(predict, feed_dict={X: test_X, y: test_y}))
            print("CNN: Epoch = %d/%d,  Batch = %d/%d, Train Accuracy = %.4f, Test Accuracy = %.4f" 
                  % (epoch + 1,num_epoch, batch_index + 1, number_batches,100*train_accuracy, 100*test_accuracy))
    
    fw_1 = sess.run(w_1)
    fw_2 = sess.run(w_2)
    fw_fc1 = sess.run(w_fc1)
    fw_fc2 = sess.run(w_fc2)
    
    fb_1 = sess.run(b_1)
    fb_2 = sess.run(b_2)
    fb_fc1 = sess.run(b_fc1)
    fb_fc2 = sess.run(b_fc2)
    
    
    np.save('./output/weight_cnn' ,[fw_1, fw_2, fw_fc1, fw_fc2, fb_1, fb_2, fb_fc1, fb_fc2])
    sess.close()
    
    

    print("CNN: Weigths and Biases saved !!")
    return    

def train_cnn_test(testX):
    
    testX = cnn_convertToVector(testX) 
    x_size = testX.shape[1] 
    
    [w_1, w_2, w_fc1, w_fc2, b_1, b_2, b_fc1, b_fc2] = np.load('./output/cnn_weight')
    
    print("CNN: Weigths and Biases restored !!")
    
     # Symbols  
    X = tf.placeholder("float32", shape=[None, x_size])
    
    yhat = cnn_ForwardPropogation(X, w_1, b_1, w_2, b_2, w_fc1, b_fc1, w_fc2, b_fc2)
    predict = tf.argmax(yhat, axis=1)
        
    testY = np.zeros(testX.shape[0], dtype = 'int')
    
    with tf.Session() as sess:
        for i in range(testX.shape[0]):
            testY[i: i+1] = sess.run(predict, feed_dict={X: testX[i: i + 1]})
        

#     with tf.Session() as sess:        
#         testY = sess.run(predict, feed_dict={X: testX})  
    return testY

In [4]:
def load_mnist():
    data_dir = '../data'

    fd = open(os.path.join(data_dir, 'train-images-idx3-ubyte'))
    loaded = np.fromfile(file=fd, dtype=np.uint8)
    trX = loaded[16:].reshape((60000, 28, 28, 1)).astype(np.float)

    fd = open(os.path.join(data_dir, 'train-labels-idx1-ubyte'))
    loaded = np.fromfile(file=fd, dtype=np.uint8)
    trY = loaded[8:].reshape((60000)).astype(np.int)

    fd = open(os.path.join(data_dir, 't10k-images-idx3-ubyte'))
    loaded = np.fromfile(file=fd, dtype=np.uint8)
    teX = loaded[16:].reshape((10000, 28, 28, 1)).astype(np.float)

    fd = open(os.path.join(data_dir, 't10k-labels-idx1-ubyte'))
    loaded = np.fromfile(file=fd, dtype=np.uint8)
    teY = loaded[8:].reshape((10000)).astype(np.int)

    trY = np.asarray(trY)
    teY = np.asarray(teY)

    perm = np.random.permutation(trY.shape[0])
    trX = trX[perm]
    trY = trY[perm]

    perm = np.random.permutation(teY.shape[0])
    teX = teX[perm]
    teY = teY[perm]

    return trX, trY, teX, teY

In [5]:
def print_digit(digit_pixels, label='?'):
    for i in range(28):
        for j in range(28):
            if digit_pixels[i, j] > 128:
                print '#',
            else:
                print '.',
        print ''

    print 'Label: ', label

In [ ]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

# Create some wrappers for simplicity
def conv2d(x, W, b, strides=1):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)


def maxpool2d(x, k=2):
    # MaxPool2D wrapper
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],
                          padding='SAME')

# Create model
def conv_net(x, weights, biases, dropout):
    # Reshape input picture
    x = tf.reshape(x, shape=[-1, 28, 28, 1])

    # Convolution Layer
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    # Max Pooling (down-sampling)
    conv1 = maxpool2d(conv1, k=2)

    # Convolution Layer
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    # Max Pooling (down-sampling)
    conv2 = maxpool2d(conv2, k=2)

    # Fully connected layer
    # Reshape conv2 output to fit fully connected layer input
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    # Apply Dropout
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output, class prediction
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out



# def conv2d_1(x, W):
#     return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

# def conv2d(x, W):
#     return tf.nn.conv2d(x, W, strides=[1, 2, 2, 1], padding='SAME')


def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

def max_pool_3x3(x):
    return tf.nn.max_pool(x, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME')

def train_cnn(trainX, trainY):
    
    '''
    Complete this function.
    '''
    
    # Network Parameters
    n_input = 784 # MNIST data input (img shape: 28*28)
    n_classes = 10 # MNIST total classes (0-9 digits)
    dropout = 0.5 # Dropout, probability to keep units
    learning_rate = 0.001
    training_iters = 200000
    batch_size = 128
    display_step = 10


    # tf Graph input
    x = tf.placeholder(tf.float32, shape=[None, 28, 28, 1])
    y = tf.placeholder(tf.float32, [None, n_classes])
    keep_prob = tf.placeholder(tf.float32) #dropout (keep probability)
    
    # Store layers weight & bias
    weights = {
        # 5x5 conv, 1 input, 32 outputs
        'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32], 0.1)),
        # 5x5 conv, 32 inputs, 64 outputs
        'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64], 0.1)),
        # fully connected, 7*7*64 inputs, 1024 outputs
        'wd1': tf.Variable(tf.random_normal([7*7*64, 1024], 0.1)),
        # 1024 inputs, 10 outputs (class prediction)
        'out': tf.Variable(tf.random_normal([1024, n_classes], 0.1))
    }

    biases = {
        'bc1': tf.Variable(tf.random_normal([32], 0.1)),
        'bc2': tf.Variable(tf.random_normal([64], 0.1)),
        'bd1': tf.Variable(tf.random_normal([1024], 0.1)),
        'out': tf.Variable(tf.random_normal([n_classes], 0.1))
    }

    # Construct model
    pred = conv_net(x, weights, biases, keep_prob)
#     y_conv = tf.nn.softmax(pred)
    
    # Define loss and optimizer
#     cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
    cost  = -tf.reduce_sum(y * tf.log(tf.clip_by_value(y_conv, 1e-10,1.0)))
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

    # Evaluate model
    correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

    y_mod = []
    for i in trainY:
        zer = [0]*10
        zer[i] = 1
        y_mod.append(zer)

    trainY_mod = np.asarray(y_mod)
    
    # Initializing the variables
    init = tf.global_variables_initializer()
    
    
    sess = tf.InteractiveSession()
    sess.run(init)
    # Launch the graph
#     with tf.Session() as sess:
#         sess.run(init)
    step = 1
    # Keep training until reach max iterations
    while step * batch_size < training_iters:
        batch_x, batch_y = trainX[step*batch_size : (step+1)*batch_size], trainY_mod[step*batch_size : (step+1)*batch_size]
        # Run optimization op (backprop)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y,
                                       keep_prob: dropout})
        if step % display_step == 0:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([cost, accuracy], feed_dict={x: batch_x,
                                                              y: batch_y,
                                                              keep_prob: 1.})
            print "Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc)
        step += 1
    print "Optimization Finished!"

    
    
#     saver = tf.train.Saver({"wc1": weights['wc1'], "wc2": weights['wc2'], "wd1": weights['wd1'], "wout": weights['out'], "bc1": biases['bc1'], "bc2": biases['bc2'], "bd1": biases['bd1'],  "bout": biases['out'],})
#     saver.save(sess, 'output/model.ckpt')
    
    var = sess.run( weights['wc1'] )
    np.save('wc1', var)
    
    var = sess.run( weights['wc2'] )
    np.save('wc2', var)
    
    var = sess.run( weights['wd1'] )
    np.save('wd1', var)
    
    var = sess.run( weights['out'] )
    np.save('wout', var)
    
    var = sess.run( biases['bc1'] )
    np.save('bc1', var)
    
    var = sess.run( biases['bc2'] )
    np.save('bc2', var)
    
    var = sess.run( biases['bd1'] )
    np.save('bd1', var)
    
    var = sess.run( biases['out'] )
    np.save('bout', var)
    
    
#     var = weights['wc1']
#     saver = tf.train.Saver({"wc1": var})
#     saver.save(sess, 'w_c1')
    
#     var = weights['wc2']
#     saver = tf.train.Saver({"wc2": var})
#     saver.save(sess, 'w_c2')
    
#     var = weights['wd1']
#     saver = tf.train.Saver({"wd1": var})
#     saver.save(sess, 'w_d1')
    
#     var = weights['out']
#     saver = tf.train.Saver({"w_out": var})
#     saver.save(sess,'w_out')
    
#     var = biases['bc1']
#     saver = tf.train.Saver({"bc1": var})
#     saver.save(sess, 'b_c1')
    
#     var = biases['bc2']
#     saver = tf.train.Saver({"bc2": var})
#     saver.save(sess, 'b_c2')
    
#     var = biases['bd1']
#     saver = tf.train.Saver({"bd1": var})
#     saver.save(sess, 'b_d1')
    
#     var = biases['out']
#     saver = tf.train.Saver({"b_out": var})
#     saver.save(sess, 'b_out')
    
   
    sess.close()
    
    
#     n_hidden = 400
#     n_input = 784
#     n_classes = 10

    
#     x = tf.placeholder(tf.float32, shape=[None, 28, 28, 1])
#     y_ = tf.placeholder(tf.int64, [None, 10])
#     y = tf.placeholder(tf.int64, [None, ])

   
    
#     W_conv1 = weight_variable([5, 5, 1, 32])
#     b_conv1 = bias_variable([32])

#     h_conv1 = tf.nn.relu(conv2d(x, W_conv1) + b_conv1)
#     h_pool1 = max_pool_3x3(h_conv1)

#     W_conv2 = weight_variable([5, 5, 32, 64])
#     b_conv2 = bias_variable([64])

#     h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
#     h_pool2 = max_pool_3x3(h_conv2)
    
#     W_fc1 = weight_variable([7 * 7 * 64, 1024])
#     b_fc1 = bias_variable([1024])

#     h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
#     h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

#     keep_prob = tf.placeholder(tf.float32)
#     h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

#     W_fc2 = weight_variable([1024, 10])
#     b_fc2 = bias_variable([10])

#     y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2
    
    
#     cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
    
#     train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
    
#     correct_prediction = tf.equal(tf.argmax(y_conv,1), y_)
#     accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
#     y_mod = []
#     for i in trainY:
#         zer = [0]*10
#         zer[i] = 1
#         y_mod.append(zer)

#     trainY_mod = np.asarray(y_mod)
    
    
#     sess = tf.InteractiveSession()
    
#     tf.global_variables_initializer().run()
    
#     epoch = 0
#     batch_size = 1000
    
#     print x.shape
#     print trainX.shape
#     print y_.shape
#     print trainY.shape
#     print trainY_mod.shape
    
#     for i in range(10):
    
#         for i in range(batch_size):
#             batch_xs, batch_ys = trainX[i*batch_size : (i+1)*batch_size], trainY_mod[i*batch_size : (i+1)*batch_size]
#             train_step.run(feed_dict={x: batch_xs, y_: batch_ys, keep_prob: 0.5})

# #             sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

    
#         train_accuracy = accuracy.eval(feed_dict={x: trainX, y: trainY, keep_prob: 0.5})

#         print 'Epoch # ', epoch+1, 'Accuracy: ', train_accuracy*100
#         epoch += 1

    
#     np.save('W_conv1', W_conv1)
#     np.save('W_conv2', W_conv2)
#     np.save('W_fc1', W_fc1)
#     np.save('W_fc2', W_fc2)
    
#     np.save('b_conv1', b_conv1)
#     np.save('b_conv2', b_conv2)
#     np.save('b_fc1', b_fc1)
#     np.save('b_fc2', b_fc2)

#     sess.close()



def test_cnn(testX):
    '''
    Complete this function.
    This function must read the weight files and
    return the predicted labels.
    The returned object must be a 1-dimensional numpy array of
    length equal to the number of examples. The i-th element
    of the array should contain the label of the i-th test
    example.
    '''
    # Network Parameters
    n_input = 784 # MNIST data input (img shape: 28*28)
    n_classes = 10 # MNIST total classes (0-9 digits)
    dropout = 0.5 # Dropout, probability to keep units
    learning_rate = 0.001
    training_iters = 200000
    batch_size = 128
    display_step = 10
    
    # tf Graph input
    x = tf.placeholder(tf.float32, shape=[None, 28, 28, 1])
    y = tf.placeholder(tf.float32, [None, n_classes])
    keep_prob = tf.placeholder(tf.float32) #dropout (keep probability)
    
    # Store layers weight & bias
    weights = {
        # 5x5 conv, 1 input, 32 outputs
        'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32], 0.1)),
        # 5x5 conv, 32 inputs, 64 outputs
        'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64], 0.1)),
        # fully connected, 7*7*64 inputs, 1024 outputs
        'wd1': tf.Variable(tf.random_normal([7*7*64, 1024], 0.1)),
        # 1024 inputs, 10 outputs (class prediction)
        'out': tf.Variable(tf.random_normal([1024, n_classes], 0.1))
    }

    biases = {
        'bc1': tf.Variable(tf.random_normal([32], 0.1)),
        'bc2': tf.Variable(tf.random_normal([64], 0.1)),
        'bd1': tf.Variable(tf.random_normal([1024], 0.1)),
        'out': tf.Variable(tf.random_normal([n_classes], 0.1))
    }
    
    
    weights['wc1'] = tf.Variable(np.load('wc1.npy'))
    
    weights['wc2'] = tf.Variable(np.load('wc2.npy'))
    
    weights['wd1'] = tf.Variable(np.load('wd1.npy'))
    
    weights['out'] = tf.Variable(np.load('wout.npy'))
    
    biases['bc1'] = tf.Variable(np.load('bc1.npy'))
    
    biases['bc2'] = tf.Variable(np.load('bc2.npy'))
    
    biases['bd1'] = tf.Variable(np.load('bd1.npy'))
    
    biases['out'] = tf.Variable(np.load('bout.npy'))
    
        
#     Construct model
    pred = conv_net(x, weights, biases, keep_prob)
    
    
    # Define loss and optimizer
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

    # Evaluate model
    correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    
    # Initializing the variables
    init = tf.global_variables_initializer()
    
    sess = tf.InteractiveSession()
    sess.run(init)
    
  
    
    out = sess.run(pred, feed_dict={x: testX, keep_prob: dropout})
    sess.close()
    
    return np.argmax(out, axis = 1)

In [6]:
# trainX, trainY, testX, testY = load_mnist()
def main():
    trainX, trainY, testX, testY = load_mnist()
    print "Shapes: ", trainX.shape, trainY.shape, testX.shape, testY.shape

#     print "\nDigit sample"
#     print_digit(trainX[1], trainY[1])
    
#     train_nn(trainX, trainY)
#     labels = test_nn(testX)
#     accuracy = np.mean((labels == testY)) * 100.0
#     print "\nDNN Test accuracy: %lf%%" % accuracy

    train_cnn_train(trainX, trainY)
    labels = train_cnn_test(testX)
    accuracy = np.mean((labels == testY)) * 100.0
    print "\nCNN Test accuracy: %lf%%" % accuracy

In [ ]:
if __name__ == '__main__':
    main()

Shapes:  (60000, 28, 28, 1) (60000,) (10000, 28, 28, 1) (10000,)
Train Size: 
(54000, 784)
Test Size: 
(6000, 784)


In [ ]:
print np.eye(10)[2]